<a href="https://colab.research.google.com/github/EmilianoLS/Data-Science/blob/main/Starbucks_KMeans/starbucks_segment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import json

In [38]:
# Json file links 
portfolio_url  = 'https://raw.githubusercontent.com/seifip/starbucks-customer-segmentation/master/data/portfolio.json'
profile_url    = 'https://raw.githubusercontent.com/seifip/starbucks-customer-segmentation/master/data/profile.json'
transcript_url = 'https://raw.githubusercontent.com/seifip/starbucks-customer-segmentation/master/data/transcript.json'

# Create dataframe
portfolio = pd.read_json(portfolio_url, lines = True)
profile = pd.read_json(profile_url, lines = True)
transcript = pd.read_json(transcript_url, lines = True)

Examinar dataframes

In [39]:
# Portfolio
# Historial de ofertas enviadas en los últimos 30 dias
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [40]:
# Profile
# Contiene información sobre la edad, fecha en la que se volvio miembro y ofertas que ha recibido
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [41]:
# Transcript
# Lista de interacciones y compras 
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


## Data Wrangling

A primera vista muchos campos no se encontrarían *normalizados*. Como mínimo, se esperaría que todos los datos se encuentren *atomizados*, motivo por el cual en la siguiente sección se buscará que en cada registro haya únicamente un solo dato (y no por ejemplo, una lista de elementos.

* Feature *channels*: se pasará a columnas y serán datos binarios
* Feature *value*: se extraerá el id de la oferta

### Obteniendo variables dummies para la feature *channels*

In [42]:
# Primero es necesario entender cuántos canales de comunicación posible existen
portfolio['channels']

0         [email, mobile, social]
1    [web, email, mobile, social]
2            [web, email, mobile]
3            [web, email, mobile]
4                    [web, email]
5    [web, email, mobile, social]
6    [web, email, mobile, social]
7         [email, mobile, social]
8    [web, email, mobile, social]
9            [web, email, mobile]
Name: channels, dtype: object

Los canales existentes son:

* email
* mobile
* social
* web

Por tal motivo se crearán 4 variables dummies de igual nombre, cada una de tipo *bool*

In [43]:
# Creo un set vacio donde se irán agregando los nombres de las columnas
# que se van a ir agregando
columns = set()
for i in range(len(portfolio)):
  for j in portfolio['channels'][i]:
    columns.add(j)

# Creo las columnas dummies vacias
for i in columns:
  portfolio[i] = 0

# Coloco 1 en aquellos canales que están presentes en la columna channels
for i in range(len(portfolio)):
  for j in portfolio['channels'][i]:
    portfolio[j][i] = 1